## Create Connections

In [2]:
%load_ext sql
import pandas as pd
import sqlite3

In [3]:
%sql sqlite:///CourseDataWarehouse.db 
conn = sqlite3.connect('CourseDataWarehouse.db')

## Attach and Create new Tables

In [5]:
%%sql
ATTACH DATABASE 'CourseData.db' AS OGDB

 * sqlite:///CourseDataWarehouse.db
Done.


[]

In [6]:
%%sql

DROP TABLE IF EXISTS CLASSMEETINGS;
CREATE TABLE CLASSMEETINGS (
    section_meetings_id INTEGER PRIMARY KEY,
    crn INTEGER,
    location_code TEXT,
    section_id INTEGER,
    catalog_id TEXT,
    course_title TEXT,
    credits INTEGER,
    prereqs TEXT,
    coreqs TEXT,
    fees INTEGER,
    attributes TEXT,
    description TEXT,
    program_code TEXT,
    title TEXT,
    section TEXT,
    cap INTEGER,
    act INTEGER,
    remaining INTEGER,
    time_codes TEXT,
    instructor_id INTEGER,
    term TEXT,
    FOREIGN KEY (location_code) REFERENCES CLASSLOCATIONS (location_code)
    FOREIGN KEY (program_code) REFERENCES CLASSPROGRAMS (program_code)
    FOREIGN KEY (time_codes) REFERENCES Time_Codes (time_codes)
    FOREIGN KEY (term) REFERENCES CLASSTERMS (term)
    FOREIGN KEY (instructor_id) REFERENCES CLASSINSTRUCTORS (instructor_id)
);
CREATE INDEX ix_section_meetings_id_alt_key on CLASSMEETINGS(section_meetings_id);

DROP TABLE IF EXISTS CLASSTERMS;
CREATE TABLE CLASSTERMS (
    term_code INTEGER PRIMARY KEY,
    term TEXT NOT NULL,
    FOREIGN KEY (term) REFERENCES CLASSMEETINGS (term)
);
CREATE INDEX ix_term_code_alt_key on CLASSTERMS(term);

DROP TABLE IF EXISTS Time_Codes;
CREATE TABLE Time_Codes (
    time_codes_id INTEGER PRIMARY KEY,
    time_codes TEXT,
    start TEXT,
    end TEXT,
    day TEXT,
    location_code TEXT,
    FOREIGN KEY (time_codes) REFERENCES CLASSMEETINGS (time_codes)
    FOREIGN KEY (location_code) REFERENCES CLASSLOCATIONS (location_code)
);
CREATE INDEX ix_time_code_id_alt_key on Time_Codes(time_codes_id);

DROP TABLE IF EXISTS CLASSLOCATIONS;
CREATE TABLE CLASSLOCATIONS (
    location_code TEXT PRIMARY KEY,
    building TEXT,
    room_number INTEGER,
    FOREIGN KEY (location_code) REFERENCES CLASSMEETINGS (location_code)
);

DROP TABLE IF EXISTS CLASSINSTRUCTORS;
CREATE TABLE CLASSINSTRUCTORS (
    instructor_id INTEGER NOT NULL PRIMARY KEY,
    instructor_name TEXT NOT NULL,
    FOREIGN KEY (instructor_id) REFERENCES CLASSMEETINGS (instructor_id)
);
CREATE INDEX ix_instructor_id_alt_key on CLASSINSTRUCTORS(instructor_name);

DROP TABLE IF EXISTS CLASSPROGRAMS;
CREATE TABLE CLASSPROGRAMS (
    program_code TEXT NOT NULL PRIMARY KEY,
    program_name TEXT NOT NULL,
    FOREIGN KEY (program_code) REFERENCES CLASSMEETINGS (program_code)
);

 * sqlite:///CourseDataWarehouse.db
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.
Done.


[]

In [7]:
%%sql

SELECT COUNT(*)
FROM (
SELECT DISTINCT s.crn, sm.location_code, s.section_id, cc.catalog_id, cc.course_title, cc.credits, cc.prereqs, cc.coreqs, 
                cc.fees, cc.attributes, cc.description, cc.program_code, s.title, s.section, s.cap, s.act, s.remaining, s.time_codes, s.instructor_name, s.term 
FROM OGDB.SECTION AS s
LEFT JOIN OGDB.CATALOGCOURSE AS cc USING (catalog_id)
LEFT JOIN OGDB.SECTIONMEETINGS AS sm USING (section_id));

 * sqlite:///CourseDataWarehouse.db
Done.


COUNT(*)
16289


In [8]:
%%sql

DELETE FROM CLASSMEETINGS;
INSERT INTO CLASSMEETINGS (crn, location_code, section_id, catalog_id, course_title, credits, prereqs, coreqs, 
                             fees, attributes, description, program_code, title, section, cap, act, remaining, time_codes, instructor_id, term)
SELECT DISTINCT s.crn, sm.location_code, s.section_id, cc.catalog_id, cc.course_title, cc.credits, cc.prereqs, cc.coreqs, 
                cc.fees, cc.attributes, cc.description, cc.program_code, s.title, s.section, s.cap, s.act, s.remaining, s.time_codes, i.instructor_id, s.term 
FROM OGDB.SECTION s
LEFT JOIN OGDB.CATALOGCOURSE cc USING (catalog_id)
LEFT JOIN OGDB.SECTIONMEETINGS sm USING (section_id)
LEFT JOIN OGDB.INSTRUCTORS i USING (instructor_name);

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
16289 rows affected.


[]

In [9]:
%%sql

DELETE FROM CLASSTERMS;
INSERT INTO CLASSTERMS (term_code, term)
SELECT DISTINCT term_code, term
FROM OGDB.TERMS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
19 rows affected.


[]

In [10]:
%%sql

DELETE FROM Time_Codes;
INSERT INTO Time_Codes (time_codes, start, end, day, location_code)
SELECT DISTINCT s.time_codes, sm.start, sm.end, sm.day, sm.location_code
FROM OGDB.SECTIONMEETINGS AS sm
LEFT JOIN OGDB.SECTION AS s USING (section_id);

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
272215 rows affected.


[]

In [11]:
%%sql

DELETE FROM CLASSLOCATIONS;
INSERT INTO CLASSLOCATIONS (location_code, building, room_number)
SELECT DISTINCT
    location_code, 
    SUBSTR(location_code,1,3), 
    SUBSTR(location_code, INSTR(location_code, " "),LENGTH(location_code) - LENGTH(INSTR(location_code, " ")))
FROM OGDB.LOCATIONS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
207 rows affected.


[]

In [12]:
%%sql

DELETE FROM CLASSINSTRUCTORS;
INSERT INTO CLASSINSTRUCTORS (instructor_id, instructor_name)
SELECT DISTINCT instructor_id, instructor_name
FROM OGDB.INSTRUCTORS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
1095 rows affected.


[]

In [13]:
%%sql

DELETE FROM CLASSPROGRAMS;
INSERT INTO CLASSPROGRAMS (program_code, program_name)
SELECT DISTINCT program_code, program_name
FROM OGDB.PROGRAMS;

 * sqlite:///CourseDataWarehouse.db
0 rows affected.
83 rows affected.


[]